# Place Cell Analysis

Interactive notebook for analyzing place cells in 2D environment navigation.

Workflow: Load data → Deconvolve → Match events → Analyze → Browse

---

**Note:** This notebook uses interactive widgets. When working over SSH, use **Jupyter Lab**:

```bash
# Remote
cd notebook && jupyter lab --no-browser --port=6006
# Local
ssh -L 6006:localhost:6006 user@remote-host
```

In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import matplotlib.pyplot as plt
from IPython.display import display
from tqdm.notebook import tqdm

from placecell.dataset import PlaceCellDataset
from placecell.notebook import browse_units, create_deconv_browser
from placecell.visualization import (
    plot_behavior_preview,
    plot_coverage,
    plot_diagnostics,
    plot_footprints,
    plot_occupancy_preview,
    plot_summary_scatter,
)

## Configuration

In [ ]:
CONFIG_PATH = project_root / "placecell/config/example_pcell_config.yaml"
DATA_PATH = Path(
    #"/mnt/data/minizero_analysis/202512round/202511_analysis_placecell/"
    "/Volumes/ProcData/minizero_analysis/202512round/202511_analysis_placecell/"
    "20251201/WL25/WL25_20251201.yaml"
)

ds = PlaceCellDataset.from_yaml(CONFIG_PATH, DATA_PATH)
print(f"Config: {CONFIG_PATH}")
print(f"Data: {DATA_PATH}")

## Unit Subset (optional)

Set `UNIT_IDS` to a list of specific unit IDs for quick iteration, or `None` for all units.

In [ ]:
UNIT_IDS = None
#UNIT_IDS = [1, 2, 3, 4, 5]

## Step 1: Load Data & Deconvolve

In [ ]:
ds.load()
ds.deconvolve(
    unit_ids=UNIT_IDS,
    progress_bar=lambda x, **kw: tqdm(x, desc="Deconvolving", **kw),
)

In [ ]:
%matplotlib widget

fig_d, controls_d = create_deconv_browser(
    ds.good_unit_ids, ds.traces, ds.S_list,
    ds.neural_fps, ds.cfg.neural.trace_name,
    time_window=ds.spatial.trace_time_window,
)
plt.show()
display(controls_d)

## Step 2: Match Events & Compute Occupancy

In [ ]:
ds.match_events()
ds.compute_occupancy()

In [ ]:
plot_behavior_preview(ds.trajectory, ds.trajectory_filtered, ds.cfg.behavior.speed_threshold)
plt.show()

In [ ]:
plot_occupancy_preview(ds.trajectory_filtered, ds.occupancy_time, ds.valid_mask, ds.x_edges, ds.y_edges)
plt.show()

In [ ]:
if ds.max_proj is not None and ds.footprints is not None:
    plot_footprints(ds.max_proj, ds.footprints)
    plt.show()
else:
    print("Max projection or footprints not available.")

## Step 3: Spatial Analysis

In [ ]:
ds.analyze_units(progress_bar=lambda x, **kw: tqdm(x, desc="Analyzing", **kw))

In [ ]:
plot_diagnostics(ds.unit_results, p_value_threshold=ds.spatial.p_value_threshold or 0.05)
plt.show()

In [ ]:
plot_summary_scatter(
    ds.unit_results,
    p_value_threshold=ds.spatial.p_value_threshold or 0.05,
    stability_threshold=ds.spatial.stability_threshold,
)
plt.show()

## Place Field Coverage

Combined place field coverage map and cumulative coverage curve.

In [ ]:
place_cell_results = ds.place_cells()
n_place_cells = len(place_cell_results)
p_thresh = ds.spatial.p_value_threshold or 0.05
n_sig = sum(1 for r in ds.unit_results.values() if r["p_val"] < p_thresh)
print(f"Significant: {n_sig} / {len(ds.unit_results)}")
print(f"Place cells (sig + stable): {n_place_cells} / {len(ds.unit_results)}")

coverage_map, n_cells_arr, coverage_frac = ds.coverage()
plot_coverage(coverage_map, n_cells_arr, coverage_frac, ds.x_edges, ds.y_edges, ds.valid_mask, n_place_cells)
plt.show()

## Interactive Cell Browser

In [ ]:
%matplotlib widget

fig, controls = browse_units(ds)
plt.show()
display(controls)

## Place Cell Browser (Significant AND Stable Only)

In [ ]:
%matplotlib widget

print(f"Place cells (sig + stable): {len(place_cell_results)} / {len(ds.unit_results)}")

if len(place_cell_results) > 0:
    fig_pc, controls_pc = browse_units(ds, unit_results=place_cell_results, place_field_threshold=0.3)
    plt.show()
    display(controls_pc)
else:
    print("No cells passed both significance and stability tests.")